# 0. Install and Import Dependencies

In [39]:
!pip install mediapipe opencv-python


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
import cv2
import mediapipe as mp
import numpy as np
import time
from collections import deque
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 1. Swipe Detection

In [42]:
def leftSwipeDetection(leftWristLocations):
    return leftWristLocations[0][0] - leftWristLocations[-1][0] > 250

def rightSwipeDetection(rightWristLocations):
    return rightWristLocations[-1][0] - rightWristLocations[0][0] > 250

In [43]:
leftWristLocations = deque(maxlen=10)
rightWristLocations = deque(maxlen=10)

In [44]:
cap = cv2.VideoCapture(0)

last_left_swipe_time = 0
last_right_swipe_time = 0
swipe_delay = 1

clothing_items = ["Tank top", "Blouse", "Tshirt", "Crop top", "Long Sleeve"]
clothing_items_index = 0

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            left_wrist_coords = tuple(np.multiply(left_wrist, [640, 480]).astype(int)) ## Normalizes Coordinates
            right_wrist_coords = tuple(np.multiply(right_wrist, [640, 480]).astype(int)) ## Normalizes Coordinates

            leftWristLocations.append(left_wrist_coords)
            rightWristLocations.append(right_wrist_coords)
            
            current_time = time.time()

            if leftSwipeDetection(leftWristLocations):
                if current_time - last_left_swipe_time > swipe_delay:
                    print("Swiped Right: index: ", clothing_items_index)
                    last_left_swipe_time = current_time
                    if clothing_items_index == 0:
                        clothing_items_index = len(clothing_items) - 1
                    else:
                        clothing_items_index -= 1
                
            if rightSwipeDetection(rightWristLocations):
                if current_time - last_right_swipe_time > swipe_delay:
                    print("Swiped Left: index: ", clothing_items_index)
                    last_right_swipe_time = current_time
                    if clothing_items_index == len(clothing_items) - 1:
                        clothing_items_index = 0
                    else:
                        clothing_items_index += 1

            # Display wrist coordinates at the wrist location
            cv2.putText(image, f"Left Wrist: ({left_wrist_coords[0]}, {left_wrist_coords[1]})", 
                        left_wrist_coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display wrist coordinates at the wrist location
            cv2.putText(image, f"Right Wrist: ({right_wrist_coords[0]}, {right_wrist_coords[1]})", 
                        right_wrist_coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

        except:
            pass
            
        
        # Draw a rectangle at the top of the screen for clothing item display
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)

        # Display clothing item name
        cv2.putText(image, 'CLOTHING ITEM', (15, 12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        
        cv2.putText(image, clothing_items[clothing_items_index], 
                    (10, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()